In [1]:
!pip install transformers
!pip install evaluate
!pip install rouge


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00


In [2]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4
DEVICE = "cuda:0"

In [3]:
MODEL.to(DEVICE)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [4]:
# Loading the data

# with open('train-v2.0.json') as f:
#     data = json.load(f)

In [5]:
# Extracting context, question, and answers from the dataset

# def prepare_data(data):
#     articles = []
    
#     for article in data["data"]:
#         for paragraph in article["paragraphs"]:
#             for qa in paragraph["qas"]:
#                 question = qa["question"]

#                 if not qa["is_impossible"]:
#                   answer = qa["answers"][0]["text"]
                
#                 inputs = {"context": paragraph["context"], "question": question, "answer": answer}

            
#                 articles.append(inputs)

#     return articles

In [6]:
# data = prepare_data(data)

# # Create a Dataframe
# data = pd.DataFrame(data)

In [7]:
# data

In [8]:
from pathlib import Path
from bs4 import BeautifulSoup
import requests
from bs4.element import Comment
import urllib.request

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def parse(url):
    body = urllib.request.urlopen(url)
    soup = BeautifulSoup(body, "html.parser")
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)
    return "\p ".join(t.strip() for t in visible_texts if len(t.strip())>0)

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

xml_file = 'test_data.xml'

output_df = pd.DataFrame(columns=['question', 'answer', 'context'])
qa_dict = {}
tree = ET.parse(xml_file)
root = tree.getroot()
for qa_pair in root.findall("QAPairs")[0]:
  qa_dict['question'] = qa_pair.find('Question').text
  qa_dict['answer'] = qa_pair.find('Answer').text
  url = root.attrib['url']
  qa_dict['context'] = parse(url)
  output_df = output_df.append(qa_dict, ignore_index=True)

In [10]:
def get_span(refined_text, answers_all):

    answers = []
    for answer in answers_all:
        answer = answer.replace("\n", " ")
        answer = answer.replace("\p"," ")
        answer = answer.strip()
        answers.append(answer)

    refined_answers = []

    spans = []
    for answer in answers:
        answer = answer.replace('-',' ')
        refined_answer = " ".join(answer.split())
        refined_answers.append(refined_answer)
        result = refined_text.find(refined_answer[:50])
        if(result!=-1):
            i, j = result, result + len(refined_answer)
            spans.append((i,j))
        else:
            spans.append((-1,-1))
    return spans, refined_text, refined_answers


text = output_df['context'].iloc[0]
answers_all = output_df['answer'].tolist()

text = text.replace("\n", " ")
text = text.replace("\p"," ")
text = text.strip()
text = text.replace('-',' ')
refined_text = " ".join(text.split())

spans, refined_text, refined_answers = get_span(refined_text, answers_all)

span_df = output_df.copy()

span_df = span_df.join(pd.DataFrame(
    {
        'span_start': -1,
        'span_end': -1,
    }, index=span_df.index
))

for i in range(len(spans)):
    span = spans[i]
    span_df["answer"].iloc[i] = refined_answers[i]
    span_df["context"].iloc[i] = refined_text
    span_df["span_start"].iloc[i] = span[0]
    span_df["span_end"].iloc[i] = span[1]   

span_df = span_df[span_df["span_start"]!=-1]
span_df = span_df[span_df["span_end"]!=-1]
span_df.dropna()

,question,answer,context,span_start,span_end
0,What is (are) Adult Acute Lymphoblastic Leukem...,Key Points Adult acute lymphoblastic leukemia ...,Skip to main content An official website of th...,1056,3702
1,What are the symptoms of Adult Acute Lymphobla...,"Signs and symptoms of adult ALL include fever,...",Skip to main content An official website of th...,1370,2067
2,How to diagnose Adult Acute Lymphoblastic Leuk...,Tests that examine the blood and bone marrow a...,Skip to main content An official website of th...,1463,3922
3,What is the outlook for Adult Acute Lymphoblas...,Certain factors affect prognosis (chance of re...,Skip to main content An official website of th...,1540,1940
4,Who is at risk for Adult Acute Lymphoblastic L...,Previous chemotherapy and exposure to radiatio...,Skip to main content An official website of th...,1281,1932
5,What are the stages of Adult Acute Lymphoblast...,"Key Points Once adult ALL has been diagnosed, ...",Skip to main content An official website of th...,9583,12131
6,What are the treatments for Adult Acute Lympho...,Key Points There are different types of treatm...,Skip to main content An official website of th...,12672,24062


In [11]:
data = span_df.copy()

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]
        
        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length", 
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)
        
        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100
        
        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [13]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [14]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(10):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)
        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    
    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1
        
    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


1/2 -> Train loss: 4.315201282501221	Validation loss: 3.910266160964966


Validation batches: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


2/2 -> Train loss: 4.22133082151413	Validation loss: 3.727677583694458


Validation batches: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


...

8/2 -> Train loss: 3.6931288838386536	Validation loss: 2.9759053885936737


Validation batches: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s]


9/2 -> Train loss: 3.617266138394674	Validation loss: 2.8880302906036377


Validation batches: 100%|██████████| 1/1 [00:00<00:00,  3.19it/s]

10/2 -> Train loss: 3.5201024055480956	Validation loss: 2.8083132028579714


In [15]:
MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_tokenizer")

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

"('qa_tokenizer/tokenizer_config.json',\n 'qa_tokenizer/special_tokens_map.json',\n 'qa_tokenizer/spiece.model',\n'qa_tokenizer/added_tokens.json',\n'qa_tokenizer/tokenizer.json')"

In [16]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
    
    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
  
    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    
    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer], 
                            references=[ref_answer])
    
        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer, 
            "Predicted Answer: ": predicted_answer, 
            "BLEU Score: ": score
        }
    else:
        return predicted_answer